### Notebook tutorial: 
- This tutorial accompanies the 2019 Datathon Competition for team 2
- Question: Does Atrial Fibrilation have an effect of in the prognosis of Pulmonary Embolism.
- Data is pulled from MIMIC III, for this competition it was hosted in google bigquery. If MIMIC is downloaded locally, set sql paths accordingly. 
- Links available at: https://mimic.physionet.org/gettingstarted/access/

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

from dateutil import parser

### Data loader class
- take path of csv files in folder, convert mimic datetime to python datetime type

In [ ]:
class Data:
    def __init__(self, path):
        csv_list = glob.glob(path)
        for csv in csv_list:
            name = os.path.basename(csv).split('.')[0].split(' ')
            if isinstance(name, list):
                name = name[0]
            setattr(self,
                    name,
                    pd.read_csv(csv,
                    parse_dates=True).fillna(0)
                   )
        self._convert_to_datetime()

    def _convert_to_datetime(self):
        for obj in dir(self):
            obj = getattr(self, obj)
            if isinstance(obj, pd.DataFrame):
                for c in obj.columns:

                    if 'time' in c.lower():

In [ ]:
dfs = Data('data/*csv')

### SQL Queries
- Note these are run on google BigQuery. Change Mimic data
- QUERY FOR ADMISSION DATA

In [ ]:

cursor.execute("""
SELECT DISTINCT PatDim.SUBJECT_ID, Admissions.HADM_ID, PatDim.GENDER, PatDim.DOB,
	PatDim.DOD, PatDim.DOD_HOSP, PatDim.DOD_SSN, PatDim.EXPIRE_FLAG,
	Admissions.DIAGNOSIS, Admissions.LANGUAGE, Admissions.ETHNICITY, Admissions.MARITAL_STATUS, Admissions.RELIGION, Admissions.INSURANCE, 
	Admissions.ADMITTIME, Admissions.DISCHTIME, Admissions.DEATHTIME,
	Admissions.ADMISSION_LOCATION, Admissions.DISCHARGE_LOCATION
	FROM `mimiciii_clinical.admissions` AS Admissions

left join `mimiciii_clinical.diagnoses_icd` AS Dx on
Admissions.HADM_ID = Dx.HADM_ID

left join `mimiciii_clinical.drgcodes` AS DrgDim on
Admissions.HADM_ID = DrgDim.HADM_ID

left join `mimiciii_clinical.patients` AS PatDim on
Admissions.SUBJECT_ID = PatDim.SUBJECT_ID

right join (select DISTINCT * from `mimiciii_derived.noteevents_metadata` 

where DESCRIPTION IN
		('CHEST CTA WITH CONTRAST',
		'CT CHEST AND ABDOMEN W/O CONTRAST',
		'CT CHEST W&W/O C',
		'CT CHEST W/CONT+RECONSTRUCTION',
		'CT CHEST W/CONTRAST',
		'CT CHEST W/CONTRAST W/ONC TABLE',
		'CT CHEST W/O CONTRAST',
		'CT CHEST W/O CONTRAST W/ONC TABLES',
		'CT STEREOTAXIS CHEST W/ CONTRAST',
		'CTA CHEST W&W/O C &RECONS',
		'CTA CHEST W&W/O C&RECONS, NON-CORONARY',
		'NLSA CT CHEST WITHOUT CONTRAST',
		'P CTA CHEST W&W/O C&RECONS, NON-CORONARY PORT')
		) AS CT ON
	Admissions.HADM_ID = CT.HADM_ID

left join `mimiciii_clinical.icustays` ICUStayDim on
Admissions.HADM_ID = ICUStayDim.HADM_ID

left join (select ECGNote.SUBJECT_ID, ECGNote.HADM_ID, ECGNote.CHARTDATE, ECGNote.CATEGORY,
	ECGNote.DESCRIPTION, ECGNote.CGID, ECGNote.ISERROR, ECGNote.TEXT
	from `mimiciii_notes.noteevents` AS ECGNote

	where CATEGORY = 'ECG') AS ECG on
	Admissions.HADM_ID = ECG.HADM_ID
  
where (Dx.ICD9_CODE LIKE "415%"
		OR DrgDim.DRG_CODE in (175, 176, 1341, 1342, 1343, 1344)
		OR (DrgDim.DRG_CODE in (175, 176)
		  AND DrgDim.DESCRIPTION != 'COMPLICATED PEPTIC ULCER'
		  AND DrgDim.DESCRIPTION != 'GASTROINTESTINAL HEMORRHAGE WITHOUT COMPLICATIONS, COMORBIDITIES'))
	AND ADMISSION_LOCATION = 'EMERGENCY ROOM ADMIT';

""")

### QUERY FOR CT NOTES
Querying for chartime, we want to get cases of patients that came for pulmonary embolism cases, and remove those who might have developed PE during the ICU stay. For this purpose we are making a ~6 hour cut-off. Those that developed PE beyond 6 hours, got PE during the ICU stay.

In [ ]:

cursor.fetchone("""

SELECT DISTINCT PatDim.SUBJECT_ID, Admissions.HADM_ID, CT.CHARTDATE, CT.CHARTTIME, CT.STORETIME, CT.CATEGORY, CT.DESCRIPTION, CT.CGID, CT.ISERROR, CT.TEXT
	FROM `mimiciii_clinical.admissions` AS Admissions

left join `mimiciii_clinical.diagnoses_icd` AS Dx on
Admissions.HADM_ID = Dx.HADM_ID

left join `mimiciii_clinical.drgcodes` AS DrgDim on
Admissions.HADM_ID = DrgDim.HADM_ID

left join `mimiciii_clinical.patients` AS PatDim on
Admissions.SUBJECT_ID = PatDim.SUBJECT_ID

right join (select DISTINCT * from `mimiciii_notes.noteevents` 

where DESCRIPTION IN
		('CHEST CTA WITH CONTRAST',
		'CT CHEST AND ABDOMEN W/O CONTRAST',
		'CT CHEST W&W/O C',
		'CT CHEST W/CONT+RECONSTRUCTION',
		'CT CHEST W/CONTRAST',
		'CT CHEST W/CONTRAST W/ONC TABLE',
		'CT CHEST W/O CONTRAST',
		'CT CHEST W/O CONTRAST W/ONC TABLES',
		'CT STEREOTAXIS CHEST W/ CONTRAST',
		'CTA CHEST W&W/O C &RECONS',
		'CTA CHEST W&W/O C&RECONS, NON-CORONARY',
		'NLSA CT CHEST WITHOUT CONTRAST',
		'P CTA CHEST W&W/O C&RECONS, NON-CORONARY PORT')
		) AS CT ON
	Admissions.HADM_ID = CT.HADM_ID

left join `mimiciii_clinical.icustays` ICUStayDim on
Admissions.HADM_ID = ICUStayDim.HADM_ID
  
where (Dx.ICD9_CODE LIKE "415%"
		OR DrgDim.DRG_CODE in (175, 176, 1341, 1342, 1343, 1344)
		OR (DrgDim.DRG_CODE in (175, 176)
		  AND DrgDim.DESCRIPTION != 'COMPLICATED PEPTIC ULCER'
		  AND DrgDim.DESCRIPTION != 'GASTROINTESTINAL HEMORRHAGE WITHOUT COMPLICATIONS, COMORBIDITIES'))
	AND ADMISSION_LOCATION = 'EMERGENCY ROOM ADMIT';

/*ICU Stays*/
SELECT DISTINCT PatDim.SUBJECT_ID, Admissions.HADM_ID,
  ICUStayDim.ICUSTAY_ID, ICUStayDim.DBSOURCE, ICUStayDim.FIRST_CAREUNIT,
  ICUStayDim.LAST_CAREUNIT, ICUStayDim.INTIME, ICUStayDim.OUTTIME,
  ICUStayDim.LOS
	FROM `mimiciii_clinical.admissions` AS Admissions

left join `mimiciii_clinical.diagnoses_icd` AS Dx on
Admissions.HADM_ID = Dx.HADM_ID

left join `mimiciii_clinical.drgcodes` AS DrgDim on
Admissions.HADM_ID = DrgDim.HADM_ID

left join `mimiciii_clinical.patients` AS PatDim on
Admissions.SUBJECT_ID = PatDim.SUBJECT_ID

right join (select DISTINCT * from `mimiciii_notes.noteevents` 

where DESCRIPTION IN
		('CHEST CTA WITH CONTRAST',
		'CT CHEST AND ABDOMEN W/O CONTRAST',
		'CT CHEST W&W/O C',
		'CT CHEST W/CONT+RECONSTRUCTION',
		'CT CHEST W/CONTRAST',
		'CT CHEST W/CONTRAST W/ONC TABLE',
		'CT CHEST W/O CONTRAST',
		'CT CHEST W/O CONTRAST W/ONC TABLES',
		'CT STEREOTAXIS CHEST W/ CONTRAST',
		'CTA CHEST W&W/O C &RECONS',
		'CTA CHEST W&W/O C&RECONS, NON-CORONARY',
		'NLSA CT CHEST WITHOUT CONTRAST',
		'P CTA CHEST W&W/O C&RECONS, NON-CORONARY PORT')
		) AS CT ON
	Admissions.HADM_ID = CT.HADM_ID
  
left join `mimiciii_clinical.icustays` ICUStayDim on
Admissions.HADM_ID = ICUStayDim.HADM_ID
  
where (Dx.ICD9_CODE LIKE "415%"
		OR DrgDim.DRG_CODE in (175, 176, 1341, 1342, 1343, 1344)
		OR (DrgDim.DRG_CODE in (175, 176)
		  AND DrgDim.DESCRIPTION != 'COMPLICATED PEPTIC ULCER'
		  AND DrgDim.DESCRIPTION != 'GASTROINTESTINAL HEMORRHAGE WITHOUT COMPLICATIONS, COMORBIDITIES'))
	AND ADMISSION_LOCATION = 'EMERGENCY ROOM ADMIT';

""")

#### QUERY FOR ECG Note
ECG events is indicative of pulmonary embolism, non-contrast cases are excluded. 

In [ ]:


cursor.execute("""
SELECT DISTINCT PatDim.SUBJECT_ID, Admissions.HADM_ID, ECGNote.CHARTDATE, ECGNote.CATEGORY,
	ECGNote.DESCRIPTION, ECGNote.CGID, ECGNote.ISERROR, ECGNote.TEXT
	FROM `mimiciii_clinical.admissions` AS Admissions

left join `mimiciii_clinical.diagnoses_icd` AS Dx on
Admissions.HADM_ID = Dx.HADM_ID

left join `mimiciii_clinical.drgcodes` AS DrgDim on
Admissions.HADM_ID = DrgDim.HADM_ID

left join `mimiciii_clinical.patients` AS PatDim on
Admissions.SUBJECT_ID = PatDim.SUBJECT_ID

right join (select DISTINCT * from `mimiciii_notes.noteevents` 

where DESCRIPTION IN
		('CHEST CTA WITH CONTRAST',
		'CT CHEST AND ABDOMEN W/O CONTRAST',
		'CT CHEST W&W/O C',
		'CT CHEST W/CONT+RECONSTRUCTION',
		'CT CHEST W/CONTRAST',
		'CT CHEST W/CONTRAST W/ONC TABLE',
		'CT CHEST W/O CONTRAST',
		'CT CHEST W/O CONTRAST W/ONC TABLES',
		'CT STEREOTAXIS CHEST W/ CONTRAST',
		'CTA CHEST W&W/O C &RECONS',
		'CTA CHEST W&W/O C&RECONS, NON-CORONARY',
		'NLSA CT CHEST WITHOUT CONTRAST',
		'P CTA CHEST W&W/O C&RECONS, NON-CORONARY PORT')
		) AS CT ON
	Admissions.HADM_ID = CT.HADM_ID

left join `mimiciii_clinical.icustays` ICUStayDim on
Admissions.HADM_ID = ICUStayDim.HADM_ID

left join (select ECGNote.SUBJECT_ID, ECGNote.HADM_ID, ECGNote.CHARTDATE, ECGNote.CATEGORY,
	ECGNote.DESCRIPTION, ECGNote.CGID, ECGNote.ISERROR, ECGNote.TEXT
	from `mimiciii_notes.noteevents` AS ECGNote

	where CATEGORY = 'ECG') AS ECGNote on
	Admissions.HADM_ID = ECGNote.HADM_ID
  
where (Dx.ICD9_CODE LIKE "415%"
		OR DrgDim.DRG_CODE in (175, 176, 1341, 1342, 1343, 1344)
		OR (DrgDim.DRG_CODE in (175, 176)
		  AND DrgDim.DESCRIPTION != 'COMPLICATED PEPTIC ULCER'
		  AND DrgDim.DESCRIPTION != 'GASTROINTESTINAL HEMORRHAGE WITHOUT COMPLICATIONS, COMORBIDITIES'))
	AND ADMISSION_LOCATION = 'EMERGENCY ROOM ADMIT';

""")

### Chart Events Rhythms

- Getting Chart Events for cohorts to diagnose Atrial Fibriliation

In [ ]:

cursor.execute("""

SELECT DISTINCT Admissions.HADM_ID, ChartEvents.ICUSTAY_ID, ChartEvents.LABEL,
  ChartEvents.CHARTTIME, ChartEvents.STORETIME, ChartEvents.CGID, ChartEvents.VALUE, ChartEvents.WARNING,
  ChartEvents.ERROR
	FROM `mimiciii_clinical.admissions` AS Admissions

left join `mimiciii_clinical.diagnoses_icd` AS Dx on
Admissions.HADM_ID = Dx.HADM_ID

left join `mimiciii_clinical.drgcodes` AS DrgDim on
Admissions.HADM_ID = DrgDim.HADM_ID

left join `mimiciii_clinical.patients` AS PatDim on
Admissions.SUBJECT_ID = PatDim.SUBJECT_ID

right join (select DISTINCT * from `mimiciii_derived.noteevents_metadata` 

where DESCRIPTION IN
		('CHEST CTA WITH CONTRAST',
		'CT CHEST AND ABDOMEN W/O CONTRAST',
		'CT CHEST W&W/O C',
		'CT CHEST W/CONT+RECONSTRUCTION',
		'CT CHEST W/CONTRAST',
		'CT CHEST W/CONTRAST W/ONC TABLE',
		'CT CHEST W/O CONTRAST',
		'CT CHEST W/O CONTRAST W/ONC TABLES',
		'CT STEREOTAXIS CHEST W/ CONTRAST',
		'CTA CHEST W&W/O C &RECONS',
		'CTA CHEST W&W/O C&RECONS, NON-CORONARY',
		'NLSA CT CHEST WITHOUT CONTRAST',
		'P CTA CHEST W&W/O C&RECONS, NON-CORONARY PORT')
		) AS CT ON
	Admissions.HADM_ID = CT.HADM_ID

left join `mimiciii_clinical.icustays` ICUStayDim on
Admissions.HADM_ID = ICUStayDim.HADM_ID

left join (select ChartEvents.*, Items.LABEL
  from `mimiciii_clinical.chartevents` AS ChartEvents
  
  left join `mimiciii_clinical.d_items` Items on
  ChartEvents.ITEMID = Items.ITEMID
  
  where Items.LABEL IN
    ('Heart Rhythm',
    'Cardiac Rhythm',
    'EKG',
    'ECG')) AS ChartEvents ON
  Admissions.HADM_ID = ChartEvents.HADM_ID

where (Dx.ICD9_CODE LIKE "415%"
		OR DrgDim.DRG_CODE in (175, 176, 1341, 1342, 1343, 1344)
		OR (DrgDim.DRG_CODE in (175, 176)
		  AND DrgDim.DESCRIPTION != 'COMPLICATED PEPTIC ULCER'
		  AND DrgDim.DESCRIPTION != 'GASTROINTESTINAL HEMORRHAGE WITHOUT COMPLICATIONS, COMORBIDITIES'))
	AND ADMISSION_LOCATION = 'EMERGENCY ROOM ADMIT';

/*Querying Drugs*/

select DISTINCT MedDim.HADM_ID, MedDim.ICUSTAY_ID,
  MedDim.STARTDATE, MedDim.ENDDATE,  
  MedDim.DRUG, MedDim.DRUG_NAME_POE, MedDim.DRUG_NAME_GENERIC from `mimiciii_clinical.prescriptions` MedDim

where
  # BETA BLOCKERS
  (((DRUG LIKE '%olol%' OR DRUG_NAME_POE LIKE '%olol%' OR DRUG_NAME_GENERIC LIKE '%olol%'
  OR DRUG LIKE '%OLOL%' OR DRUG_NAME_POE LIKE '%OLOL%' OR DRUG_NAME_GENERIC LIKE '%OLOL%'
  OR DRUG LIKE '%alol%' OR DRUG_NAME_POE LIKE '%alol%' OR DRUG_NAME_GENERIC LIKE '%alol%'
  OR DRUG LIKE '%ALOL%' OR DRUG_NAME_POE LIKE '%ALOL%' OR DRUG_NAME_GENERIC LIKE '%ALOL%')
  AND (DRUG NOT LIKE '%Timolol%' AND DRUG_NAME_POE NOT LIKE '%Timolol%' AND DRUG_NAME_GENERIC NOT LIKE '%Timolol%'
  AND DRUG NOT LIKE '%timolol%' AND DRUG_NAME_POE NOT LIKE '%timolol%' AND DRUG_NAME_GENERIC NOT LIKE '%timolol%'))
  # DIGOXIN
  OR (DRUG LIKE '%Digoxin%' OR DRUG_NAME_POE LIKE '%Digoxin%' OR DRUG_NAME_GENERIC LIKE '%Digoxin%'
  AND (DRUG != 'Indigotindisulfonate Sodium' AND DRUG_NAME_POE != 'Indigotindisulfonate Sodium' AND DRUG_NAME_GENERIC != 'Indigotindisulfonate Sodium'))
  # AMIORODARONE
  OR DRUG LIKE '%Amio%'
  # Diltiazem
  OR DRUG LIKE '%Dilt%' OR DRUG_NAME_POE LIKE '%Dilt%' OR DRUG_NAME_GENERIC LIKE '%Dilt%'
  OR DRUG LIKE '%DILT%' OR DRUG_NAME_POE LIKE '%DILT%' OR DRUG_NAME_GENERIC LIKE '%DILT%'
  OR DRUG LIKE '%dilt%' OR DRUG_NAME_POE LIKE '%dilt%' OR DRUG_NAME_GENERIC LIKE '%dilt%'
  # Verampamil
  OR (DRUG LIKE '%Vera%' OR DRUG_NAME_POE LIKE '%Vera%' OR DRUG_NAME_GENERIC LIKE '%Vera%'
  AND (DRUG != 'Provera' AND DRUG_NAME_POE != 'Provera' AND DRUG_NAME_GENERIC != 'Provera')))
  AND MedDim.HADM_ID in
  
""")


(139788,
  182135,
  172454,
  165934,
  144159,
  110275,
  100826,
  164098,
  129744,
  182573,
  199898,
  154639,
  142149,
  114241,
  107530,
  130079,
  128160,
  198911,
  195826,
  152188,
  167021,
  138421,
  158183,
  196065,
  106481,
  103175,
  167558,
  161020,
  185409,
  109340,
  188192,
  181683,
  142100,
  178274,
  113350,
  138803,
  119596,
  127242,
  102942,
  110568,
  134311,
  175238,
  199207,
  125277,
  178211,
  151145,
  131893,
  179968,
  109361,
  100525,
  124391,
  125077,
  151731,
  196982,
  197611,
  123815,
  174899,
  191645,
  169308,
  172466,
  125303,
  169182,
  169525,
  144752,
  124627,
  172672,
  163318,
  157722,
  103931,
  173330,
  182280,
  131239,
  144709,
  125831,
  142254,
  181593,
  187907,
  143994,
  183007,
  122694,
  118736,
  194201,
  113272,
  168854,
  190004,
  115843,
  100187,
  109789,
  139698,
  146274,
  167200,
  142220,
  180931,
  196923,
  148135,
  186563,
  135816,
  154406,
  122008,
  128939,
  146001,
  114875,
  176465,
  189800,
  133123,
  107449,
  115789,
  168926,
  123997,
  198939,
  171444,
  102486,
  164491,
  164045,
  127372,
  124978,
  122833,
  153465,
  140567,
  129591,
  109177,
  197164,
  136113,
  125994,
  135402,
  124090,
  172231,
  130339,
  110565,
  187245,
  191928,
  176069,
  149772,
  153168,
  178987,
  113352,
  195738,
  131822,
  134829,
  176952,
  100575,
  118985,
  166576,
  175356,
  167545,
  100751,
  114608,
  185178,
  174576,
  118633,
  142398,
  101266,
  120230,
  171678,
  100584,
  190685,
  115245,
  111685,
  185837,
  138477,
  154613,
  186572,
  158616,
  181085,
  130727,
  177514,
  136084,
  194728,
  118521,
  193169,
  109760,
  108685,
  147705,
  120442,
  106011,
  193351,
  100030,
  135931,
  116814,
  192838,
  128551,
  121230,
  141664,
  148647,
  174449,
  110118,
  189434,
  197327,
  148971,
  166975,
  187036,
  190669,
  133462,
  130729,
  145785,
  149505,
  175438,
  185562,
  194657,
  188684,
  137650,
  163354,
  190479,
  122767,
  192255,
  181819,
  159989,
  114600,
  129723,
  152022,
  183460,
  199082,
  137236,
  130976,
  153323,
  140695,
  172970,
  143957,
  185663,
  176097,
  128820,
  137954,
  104178,
  159775,
  196214,
  145248,
  114966,
  119486,
  152544,
  171645,
  103536,
  165555,
  102497,
  138107,
  142459,
  112708,
  150709,
  185880,
  177936,
  170682,
  123562,
  155766,
  179295,
  145132,
  133292,
  104849,
  105316,
  106345,
  106598,
  199909,
  170174,
  126687,
  156055,
  135487,
  147116,
  130430,
  162870,
  119437,
  153476,
  177633,
  140501,
  119703,
  147424,
  193135,
  193966,
  134007,
  125835,
  120501,
  173941,
  171996,
  175366,
  180029,
  143170,
  109946,
  155003,
  197965,
  198299,
  157955,
  194762,
  147692,
  142391,
  131236,
  138680,
  130076,
  163302,
  134599,
  189465,
  168087,
  186629,
  108482,
  179237,
  189583,
  138790,
  158130,
  188536,
  128091,
  176777,
  141048,
  157336,
  123375,
  192572,
  166585,
  158050,
  170681,
  128051,
  124867,
  152365,
  141081,
  174684,
  141268,
  196212,
  160465,
  117365,
  182868,
  187951,
  187617,
  186470,
  153119,
  153102,
  131885,
  141787,
  113509,
  151364,
  106296,
  102735,
  115961,
  171451,
  124038,
  140887,
  137032,
  193045,
  188843,
  187032,
  133463,
  119804,
  163976,
  129648,
  118574,
  103629,
  130358,
  122659,
  181599,
  111057,
  194869,
  131143,
  168118,
  153468,
  133404,
  104850,
  157398,
  194125,
  146115,
  103997,
  162478,
  197959,
  128573,
  128912,
  131639,
  179681,
  170978,
  132585,
  168225)

#### Importing Data From CSV

In [3]:


admissions = pd.read_csv('Admissions.csv', low_memory=False, index_col=None)
icu = pd.read_csv('ICU_Stays.csv', low_memory=False, index_col=None)
ct_notes = pd.read_csv('CT Notes.csv', low_memory=False, index_col=None)
drugs = pd.read_csv('drugs.csv', low_memory=False, index_col=None)
elixhauser_score = pd.read_csv('Elixhauser Scores.csv', low_memory=False, index_col=None)
elixhauser_comorbidities = pd.read_csv('Elixhauser Comorbidities.csv', low_memory=False, index_col=None)

#### Converting to Date:Time

In [11]:
icu['INTIME'] = pd.to_datetime(icu['INTIME'])

#### Checking that all patients had ICU stays

In [5]:
icu_stay_counts = icu[['HADM_ID', 'ICUSTAY_ID']].groupby('HADM_ID').count()
no_icu = list(icu_stay_counts[icu_stay_counts['ICUSTAY_ID'] == 0].index)
first_icu_times = icu[['HADM_ID', 'INTIME']].groupby('HADM_ID').min()

#### Dropping patients without ICU stays

In [6]:
admissions = admissions[~admissions['HADM_ID'].isin(no_icu)]
icu = icu[~icu['HADM_ID'].isin(no_icu)]

#### Selecting first ICU stay

In [15]:
icu = icu.loc[icu.groupby('HADM_ID')['INTIME'].idxmin()]

#### Merging ICU stay on admissions

In [8]:
admissions = admissions.merge(right=icu, how='left', on='HADM_ID')\
   .merge(right=elixhauser_score, how='left', left_on='HADM_ID', right_on='hadm_id')\
   .merge(right=elixhauser_comorbidities, how='left', left_on='HADM_ID', right_on='hadm_id')

#### Dropping CT Notes without contrast

In [9]:
ct_notes = ct_notes[ct_notes['DESCRIPTION'] != 'CT CHEST W/O CONTRAST']

####  Finding admission to ICU time and joining to CT note DataFrame

In [10]:

ct_notes = ct_notes.merge(right=admissions[['HADM_ID', 'INTIME']], on='HADM_ID')
ct_notes['INTIME'] = pd.to_datetime(ct_notes['INTIME'])
ct_notes['CHARTTIME'] = pd.to_datetime(ct_notes['CHARTTIME'])
ct_notes['CTTIMEDELTA'] = (ct_notes.INTIME-ct_notes.CHARTTIME).astype('timedelta64[h]')